# 🎨 AI Comic Book Generator - Colab Pro Edition

This notebook runs the AI Comic Generator **completely FREE** using local Ollama models and includes a **Live HITL Dashboard** for real-time control.

**Features:**
- 🧠 **Dual-Model Logic:** Llama 3.1 8B for reasoning, Llama 3.2 3B for speed.
- 🖼️ **SDXL Integration:** High-quality image generation via Diffusers.
- 🖥️ **HITL Dashboard:** Live preview and edit scenes/characters mid-process.
- 🔄 **Checkpoint System:** Resume from where you left off if Colab disconnects.

In [ ]:
# @title 🛠️ 1. Initial Setup (Run Once)
# @markdown Installs Ollama, Node.js, and clones the repository.
REPO_URL = "https://github.com/yogesh-dixit-dev/comic-generator.git" # @param {type:"string"}
BRANCH = "master" # @param {type:"string"}

import os
import sys
import time
import subprocess

def run_command(cmd, desc):
    print(f"📦 {desc}...")
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"❌ Error: {result.stderr}")
    return result

# Install dependencies
os.system("apt-get update -qq && apt-get install -y -qq zstd curl")

# Install Node.js (required for Dashboard)
print("🟢 Installing Node.js...")
os.system("curl -fsSL https://deb.nodesource.com/setup_20.x | bash - && apt-get install -y nodejs")

# Install Ollama
print("🟢 Installing Ollama...")
os.system("curl -fsSL https://ollama.com/install.sh | sh")

# Start Ollama server in background
print("🚀 Starting Ollama server...")
os.system("nohup ollama serve > ollama.log 2>&1 &")

time.sleep(5) # Wait for server

# Pull models
print("⬇️ Downloading Models (Llama 3.1 & 3.2)...")
os.system("ollama pull llama3.1")
os.system("ollama pull llama3.2")

# Clone repo
if not os.path.exists("comic-gen"):
    run_command(f"git clone -b {BRANCH} {REPO_URL} comic-gen", "Cloning Repository")

os.chdir("/content/comic-gen")
run_command("pip install -q -r requirements.txt", "Installing Python Dependencies")

print("🟢 Installing Dashboard Dependencies (this may take a minute)...")
os.chdir("/content/comic-gen/infrastructure/web")
os.system("npm install -s")
os.chdir("/content/comic-gen")

print("\n✅ Core Setup Complete!")

In [ ]:
# @title 🖥️ 2. Launch HITL Dashboard
# @markdown Starts the backend server and frontend dashboard. Provides multiple access methods.
import os
import subprocess
import time
import socket
import sys

def is_port_in_use(port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

# Ensure we are in the right directory
os.chdir("/content/comic-gen")

print("🚀 Starting Backend Server (Port 8000)...")
if not is_port_in_use(8000):
    os.system(f"PYTHONPATH=. nohup {sys.executable} infrastructure/server/app.py > server.log 2>&1 &")
    time.sleep(3)

print("🚀 Starting Frontend Dashboard (Port 3000)...")
if not is_port_in_use(3000):
    os.chdir("infrastructure/web")
    os.system("nohup npm run dev -- --port 3000 --host > dashboard.log 2>&1 &")
    os.chdir("/content/comic-gen")
    time.sleep(3)

print("\n🌍 --- ACCESS METHOD 1: COLAB PROXY (Recommended) ---")
try:
    from google.colab.output import eval_js
    proxy_url = eval_js("google.colab.kernel.proxyPort(3000)")
    print(f"🔗 Dashboard URL: {proxy_url}")
except:
    print("⚠️ Colab proxy unavailable. Use Method 2.")

print("\n🔗 --- ACCESS METHOD 2: LOCALTUNNEL (Fallback) ---")
print("Checking for Localtunnel...")
res = subprocess.run("lt --version", shell=True, capture_output=True)
if res.returncode != 0:
    os.system("npm install -g localtunnel > /dev/null 2>&1")

print("Starting tunnel...")
os.system("nohup lt --port 3000 > tunnel.log 2>&1 &")
time.sleep(2)
if os.path.exists("tunnel.log"):
    with open("tunnel.log", "r") as f:
        print(f"🔗 Public URL: {f.read().strip()}")
        # Get local IP for Localtunnel bypass
        import requests
        try:
            local_ip = requests.get('https://ipv4.icanhazip.com').text.strip()
            print(f"🔑 Tunnel Password (if prompted): {local_ip}")
        except:
            pass

print("\n---------------------------\n")
print("💡 Troubleshooting:")
print("1. If Method 1 fails with '404', wait 10s and refresh.")
print("2. If the UI shows 'Backend unreachable', check server.log:")
print("!tail -n 20 server.log")

In [ ]:
# @title 📜 3. Progress Tracking Helper
import subprocess
import re
from tqdm.notebook import tqdm

def run_pipeline_with_progress(command, phase_name):
    print(f"🚀 Starting {phase_name} Phase...")
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    
    pbar = tqdm(total=100, desc=phase_name, unit="%")
    last_progress = 0
    
    for line in iter(process.stdout.readline, ''):
        if "[PROGRESS]" in line:
            match = re.search(r"\[PROGRESS\] (\d+)%", line)
            if match:
                progress = int(match.group(1))
                pbar.update(progress - last_progress)
                last_progress = progress
        else:
            print(line, end='')
            
    pbar.n = 100
    pbar.refresh()
    process.wait()
    if process.returncode == 0:
        print(f"\n✅ {phase_name} complete!")
    else:
        print(f"\n❌ {phase_name} failed with code {process.returncode}")

In [ ]:
# @title 🎬 4. Run Pipeline: Planning Phase
# @markdown Narrates the story, designs characters, and plans panels. 
# @markdown **Check the Dashboard to approve/edit!**
STORY_TEXT = "In a futuristic Tokyo, a cybernetic samurai named Kaito discovers a lost biological cat in the neon slums. He must protect it from rogue drones while seeking a safe haven." # @param {type:"string"}
INPUT_FILE = "story.txt"

with open(INPUT_FILE, "w") as f:
    f.write(STORY_TEXT)

cmd = f"python3 src/main.py --input '{INPUT_FILE}' --phase plan --colab"
run_pipeline_with_progress(cmd, "Planning")

In [ ]:
# @title 🎨 5. Run Pipeline: Drawing Phase
# @markdown Generates high-quality images using SDXL. Run this AFTER approving the plans in the dashboard.
INPUT_FILE = "story.txt"

cmd = f"python3 src/main.py --input '{INPUT_FILE}' --phase draw --colab"
run_pipeline_with_progress(cmd, "Drawing")

In [ ]:
# @title 📦 6. Download Results
from google.colab import files
import os

if os.path.exists('output'):
    os.system("zip -r comic_output.zip output")
    files.download('comic_output.zip')
    print("✅ Download started!")
else:
    print("⚠️ No output folder found.")